In [13]:
import random
def generate(numTables, numQueries, queryTypes,maxnumoftablesperquery):
    lines = []
    qtypes = []
    nqtypes = len(queryTypes)
    visit = 0
    for i in range(numQueries):
        temp = random.randint(1,nqtypes)
        qtypes.append(queryTypes[temp-1])
        temp = random.randint(1,maxnumoftablesperquery)
        templine = []
        if(visit==0):
            start = 1
            end = 10
            visit=1
        else:
            start=11
            end=numTables
            visit=0
        for j in range(temp):
            templine.append(random.randint(1,numTables))
        lines.append(list(set(templine)))
    #  print(lines)
    # print(qtypes)
    return [lines, qtypes]

In [14]:
def gene():
    f = open("queries1.sql","r")

    #dataset - list of lists, to hold all the table names used in each query
    dataset=[]

    for line in f:
        #Parsing select queries
        if(line.strip()=="select"):
            while(line.strip()!="from"):
                line = f.readline()

            #query- List which will hold the tables used in a query
            query =[]
            line = f.readline()

            if(line.rstrip().split()[0] != "("):
                while(line!="where" and line!="group by" and line!="order by" and line!= "on" and line != ")"):
                    if("join" in line):
                        query.append(line.split("join")[0].split()[0])
                        query.append(line.split("join")[1].split()[0])
                        break;# for now the join queries are not handled
                    else:
                        line = line.rstrip().split(',')[0].split()[0]
                        if(line!="where" and line!="group" and line!="order" and line!= "on" and line != ")"):
                            query.append(line)
                            line = f.readline()
                        else:break
                if(len(query)!=0):
                    dataset.append(query)
            elif(line.rstrip() == "("):
                continue
    return dataset
                   

        
        

In [15]:
def generateInitFreq(lines,qtypes):
    s_freq =  0
    u_freq = 0
    i_freq = 0
    d_freq =0
    for i in qtypes:
        if i=='S':
            s_freq+=1
        elif i == 'U':
            u_freq+=1
        elif i =='D':
            d_freq+=1
        elif i=='I':
            i_freq+=1
    s_freq/=len(qtypes)
    u_freq/=len(qtypes)
    d_freq/=len(qtypes)
    i_freq/=len(qtypes)
    schema = []
    #if schema not given then
    for i in lines:
        schema.extend(i)
    schema  = list(set(schema))
    #print(schema)
    freq= []
    for i in schema:
        tfreq = 0
        for j in range(0,len(lines)):
            if i in lines[j]:
                if qtypes[j] == 'S':
                    tfreq+=s_freq
                if qtypes[j] == 'U':
                    tfreq+=u_freq
                if qtypes[j] == 'I':
                    tfreq+=i_freq
                if qtypes[j] == 'D':
                    tfreq+=d_freq
        freq.append(tfreq)
    return [schema, freq]

In [16]:
import numpy
def getRankingOrder(schema, freq):
        freq = numpy.array(freq)
        sorted_indexes = numpy.argsort(freq)[::-1][:len(freq)]
        #print(sorted_indexes)
        sorted_list = []
        for i in sorted_indexes:
            sorted_list.append(schema[i])
        return(sorted_list)

In [17]:
import math
def runAlgo(lines, freq, schema, alpha, max_iter, defined_error):
    #print(lines)
    mat = []
    for i in range(0,len(schema)):
        temp = []
        for j in range(0,len(schema)):
            temp.append(0)
        mat.append(temp)
    for i in range(0,len(schema)):
        st = schema[i]
        for j in range(0,len(schema)):
            if i==j:
                continue
            else:
                dt = schema[j]
                score = 0
                level = -1
                for k in range(0,len(lines)):
                    if st in lines[k]:
                        if dt in lines[k]:
                            score+=2
                        level = 0
                    elif level>-1:
                        if dt in lines[k]:
                            if(level==0):
                                score+=1;
                            else:
                                score+=1/(level)
                        level+=1
                mat[i][j] = score
    #print(mat)
    for i in range(0, len(schema)):
        tsum =0
        for j in range(0, len(schema)):
            tsum+=mat[i][j]
        for j in range(0, len(schema)):
            mat[i][j] = mat[i][j]/tsum
    #print(mat)
    lfreq = freq
    for _ in range(max_iter):
        nfreq=[]
        for i in range(0,len(schema)):
            gotpr = 0
            for j in range(0,len(schema)):
                gotpr+= lfreq[j]*mat[j][i]
            nfreq.append((1-alpha) + alpha*(gotpr))
        error = sum([abs(lfreq[n] - nfreq[n]) for n in range(0,len(schema))])
        #print(error)
        if(error<defined_error):
            break
        #print(nfreq)
        lfreq = nfreq
    return nfreq

In [18]:
def addBias(lines, toplist, biaslist, bias,maxinsertion):
    biaslen = len(biaslist)
    tempbias = 0
    lines_bias =[]
    tcount=0
    for i in range(0,len(lines)):
        templist=lines[i]
        if(len(list(set(lines[i])&set(toplist)))>0):
            if(tempbias == 0):
                insert  = random.randint(1, maxinsertion)
                for _ in range(insert):
                    templist.append(biaslist[random.randint(0, biaslen-1)])
                tcount+=1
            tempbias+=1
            if(tempbias == bias):
                tempbias=0
        lines_bias.append(list(set(templist)))
    print('queries effected:' ,tcount)
    return lines_bias

In [19]:
def splitdata(data,qtypes):
    size = len(data)
    split = int(0.7*size)
    train = data[:split]
    test= data[split:]
    train_q =  qtypes[:split]
    test_q = qtypes[split:]
    return train, test,train_q,test_q

In [20]:
def splitdata1(data):
    size = len(data)
    split = int(0.7*size)
    train = data[:split]
    test= data[split:]
    return train, test

In [21]:
#Testing-- measure 1
def computePerformance(ranking,test_data,top):
    top_n = ranking[:int(top*len(ranking))]
    performance= 0
    print(top_n)
    for i in test_data:
        inclusion= 1
        for j in i:
            if j not in top_n:
                inclusion = 0
                #print(j)
                break
        if inclusion is 1:
            performance+=1
    print(performance/len(test_data))
    return (performance/len(test_data))

In [22]:
def ComputeIntersectionPerformance(train_output,test_output):
    #Testing -- measure 2
    top_n_train = train_output[:int(0.4*len(train_output))]
    top_n_test = test_output[:int(0.4*len(test_output))]
    print(len(set(top_n_train).intersection(top_n_test)))
    print(len(top_n_train),len(top_n_test))

In [27]:
def biasAlgo(top,numTables, numQueries, queryTypes,maxnumoftablesperquery, fromtop,totop , frombias, tobias, alpha, max_iter, defined_error,bias, inserttablebias):
    [lines_first, qtypes] = generate(numTables, numQueries, queryTypes,maxnumoftablesperquery)
    #print(lines_first, qtypes)
    
    train_lines_first,test_lines_first = splitdata1(lines_first)
    
    [schema, train_freq_first] = generateInitFreq(train_lines_first, qtypes)
    #print(lines_first)
    #print(train_freq_first)
    print("Initial Frequency Ranking Order:")
    train_ranking = getRankingOrder(schema,train_freq_first)
    print(train_ranking)
    new_train_freq_first = runAlgo(train_lines_first, train_freq_first, schema,alpha, max_iter, defined_error)
    print("Initial Frequency Ranking Order after Algo:")
    ranking_after_algo = getRankingOrder(schema, new_train_freq_first)
    print(ranking_after_algo)
    lines_bias = addBias(lines_first, train_ranking[fromtop:totop], train_ranking[frombias:tobias], bias, inserttablebias)
    #print(lines_bias)
    
    train_lines_bias,test_lines_bias = splitdata1(lines_bias)
    
    [schema, train_freq_bias] = generateInitFreq(train_lines_bias, qtypes)
    #print(freq_bias)
    print("Biased Frequency Ranking Order:")
    ranking_freq_biased = getRankingOrder(schema, train_freq_bias)
    print(ranking_freq_biased)
    #print(lines_bias)
    #print(lines_first)
    new_train_freq_bias = runAlgo(train_lines_bias, train_freq_bias, schema, alpha, max_iter, defined_error)
    print("Biased Frequency Ranking Order After Algo:")
    ranking_after_algo_biased = getRankingOrder(schema, new_train_freq_bias)
    print(ranking_after_algo_biased)
    
    ########################################################################################
    #Calculating performance measure 1
    ########################################################################################
    
    print("\n\nperformance + bias + algo")
    #algo_p = computePerformance(ranking_after_algo_biased,test_lines_bias,top)
    
    print("performance + bias + freq")
    #freq_p = computePerformance(ranking_freq_biased,test_lines_bias,top)
    print("performance + algo")
    algo_p =computePerformance(ranking_after_algo,test_lines_first,top)
    print("performance + freq")
    freq_p = computePerformance(train_ranking,test_lines_first,top)
    return algo_p,(freq_p)
    ########################################################################################

In [28]:
def biasAlgo1(numTables, numQueries, queryTypes,maxnumoftablesperquery, fromtop,totop , frombias, tobias, alpha, max_iter, defined_error,bias, inserttablebias):
    [lines_first, qtypes] = generate(numTables, numQueries, queryTypes,maxnumoftablesperquery)
    
    [schema0, freq] = generateInitFreq(lines_first,qtypes)
    
    train_lines_first,test_lines_first,train_qtypes,test_qtypes = splitdata(lines_first,qtypes)
    
    [schema, train_freq_first] = generateInitFreq(train_lines_first, train_qtypes)
    [schema1, test_freq_first] = generateInitFreq(test_lines_first, test_qtypes)
    
    overall_ranking = getRankingOrder(schema0,freq)
    train_ranking = getRankingOrder(schema,train_freq_first)
    test_ranking = getRankingOrder(schema,test_freq_first)
    
    new_train_freq_first = runAlgo(train_lines_first, train_freq_first, schema,alpha, max_iter, defined_error)
    new_test_freq_first = runAlgo(test_lines_first, test_freq_first, schema,alpha, max_iter, defined_error)
    
    train_ranking_after_algo = getRankingOrder(schema, new_train_freq_first)
    test_ranking_after_algo = getRankingOrder(schema, new_test_freq_first)
    
    lines_bias = addBias(lines_first, overall_ranking[fromtop:totop], overall_ranking[frombias:tobias], bias, inserttablebias)
    
    train_lines_bias,test_lines_bias,train_bias_qtypes,test_bias_qtypes = splitdata(lines_bias,qtypes)
    
    [schema, train_freq_bias] = generateInitFreq(train_lines_bias, train_bias_qtypes)
    [schema1, test_freq_bias] = generateInitFreq(test_lines_bias, test_bias_qtypes)
    
    train_ranking_freq_biased = getRankingOrder(schema, train_freq_bias)
    test_ranking_freq_biased = getRankingOrder(schema, test_freq_bias)
    
    new_train_freq_bias = runAlgo(train_lines_bias, train_freq_bias, schema, alpha, max_iter, defined_error)
    new_test_freq_bias = runAlgo(test_lines_bias, test_freq_bias, schema, alpha, max_iter, defined_error)

    train_ranking_after_algo_biased = getRankingOrder(schema, new_train_freq_bias)
    test_ranking_after_algo_biased = getRankingOrder(schema, new_test_freq_bias)
    
    ########################################################################################
    #Calculating performance measure 1
    ########################################################################################
    
    print("\n\nperformance + bias + algo")
    computePerformance(train_ranking_after_algo_biased,test_lines_bias)
    print("performance + bias + freq")
    computePerformance(train_ranking_freq_biased,test_lines_bias)
    print("performance + algo")
    computePerformance(train_ranking_after_algo,test_lines_first)
    print("performance + freq")
    computePerformance(train_ranking,test_lines_first)
    
    ########################################################################################
    
    
    ########################################################################################
    #Calculating performance measure 1
    ########################################################################################
    
    print("\n\nperformance + bias + algo")
    ComputeIntersectionPerformance(train_ranking_after_algo_biased,test_ranking_after_algo_biased)
    print("performance + bias + freq")
    ComputeIntersectionPerformance(train_ranking_freq_biased,test_ranking_freq_biased)
    print("performance + algo")
    ComputeIntersectionPerformance(train_ranking_after_algo,test_ranking_after_algo)
    print("performance + freq")
    ComputeIntersectionPerformance(train_ranking,test_ranking)
    
    ########################################################################################

In [29]:
numTables = 50
numQueries = 10000
queryTypes = ['S','D','I','U']
maxnumoftablesperquery = 7
fromtop=0
totop=10
frombias=46
tobias=50
alpha = 0.85
max_iter = 500
defined_error = 0.00001
bias=20
inserttablebias=1

#biasAlgo(numTables, numQueries, queryTypes,maxnumoftablesperquery,fromtop,totop , frombias, tobias,alpha, max_iter, defined_error, bias, inserttablebias)



In [112]:
biasAlgo1(numTables, numQueries, queryTypes,maxnumoftablesperquery,fromtop,totop , frombias, tobias,alpha, max_iter, defined_error, bias, inserttablebias)


queries effected: 250


performance + bias + algo
[5, 4, 2, 3, 7, 1, 6, 9, 8, 10, 13, 32, 22, 40, 24, 48, 31, 26, 30, 19]
0.5086666666666667
performance + bias + freq
[4, 5, 2, 3, 7, 6, 1, 8, 9, 10, 13, 40, 32, 22, 24, 48, 26, 31, 34, 19]
0.509
performance + algo
[5, 4, 2, 3, 7, 1, 6, 9, 8, 10, 13, 32, 40, 24, 48, 31, 30, 19, 34, 21]
0.499
performance + freq
[4, 5, 2, 3, 7, 6, 1, 8, 9, 10, 13, 40, 32, 24, 48, 31, 34, 19, 12, 20]
0.499


performance + bias + algo
13
20 20
performance + bias + freq
13
20 20
performance + algo
13
20 20
performance + freq
12
20 20


In [30]:
import sys
import os
import matplotlib.pyplot as plt
import matplotlib

x=[]
y=[]
z=[]
for top in range(10,110,10):
    #print(top)
    algo_p,freq_p =biasAlgo(top/100,numTables, numQueries, queryTypes,maxnumoftablesperquery,fromtop,totop , frombias, tobias,alpha, max_iter, defined_error, bias, inserttablebias)
    x.append(float(top));
    y.append(float(algo_p));
    z.append(float(freq_p));
    plt.figure()
    matplotlib.style.use('ggplot')
    plt.plot(x, y,x,z)
    #plt.plot(x,z,'g')
    #plt.plot(x, y,'r',label="Algorithm performance")
    #plt.plot(x,z,'g',label="Frequency performance")
    plt.title("Performance")
    plt.xlabel("top n")
    plt.ylabel("performance measure 1")
    #plt.legend()
    #plt.hold(False)
    #plt.axis([0,1,0,1])
    #plt.show()
    plt.savefig("performanceMeasure11.png")
    plt.clf()
    plt.close()

Initial Frequency Ranking Order:
[36, 8, 32, 22, 1, 35, 7, 9, 46, 38, 28, 2, 17, 47, 49, 31, 48, 42, 37, 24, 10, 12, 3, 43, 44, 14, 25, 4, 16, 45, 33, 15, 23, 34, 18, 5, 19, 41, 13, 40, 39, 20, 26, 29, 21, 30, 11, 50, 6, 27]
Initial Frequency Ranking Order after Algo:
[36, 8, 22, 32, 1, 7, 35, 9, 38, 47, 2, 28, 46, 31, 42, 17, 48, 49, 37, 43, 14, 16, 24, 12, 3, 10, 33, 34, 4, 23, 25, 19, 45, 5, 44, 39, 41, 13, 15, 26, 40, 18, 29, 30, 21, 20, 50, 11, 6, 27]
queries effected: 286
Biased Frequency Ranking Order:
[36, 8, 32, 22, 1, 35, 7, 9, 46, 38, 28, 2, 6, 17, 11, 47, 49, 31, 50, 48, 42, 37, 24, 10, 12, 3, 43, 44, 14, 25, 4, 16, 45, 33, 15, 23, 34, 18, 5, 19, 41, 13, 40, 39, 27, 20, 26, 29, 21, 30]
Biased Frequency Ranking Order After Algo:
[36, 8, 22, 32, 1, 7, 35, 9, 38, 11, 28, 47, 46, 2, 6, 31, 42, 50, 17, 48, 49, 37, 43, 14, 24, 16, 12, 10, 3, 33, 34, 4, 23, 25, 19, 45, 5, 39, 44, 41, 13, 15, 27, 26, 40, 18, 29, 30, 20, 21]


performance + bias + algo
performance + bias + freq
perf